In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
n_collect = 25
coverage0 = 0.2

folders = [
#     'dy_hi', 
#     'dy_lo',
    'dy_ladd'
]

In [3]:
np.concatenate([np.linspace(0, 0.5, 6), np.linspace(0.5, 1, 11)[1:], np.linspace(1, 2, 11)[1:]])

array([0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75,
       0.8 , 0.85, 0.9 , 0.95, 1.  , 1.1 , 1.2 , 1.3 , 1.4 , 1.5 , 1.6 ,
       1.7 , 1.8 , 1.9 , 2.  ])

In [4]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 0.8, 33), tp="VulACF"):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)
            
            idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Plain') or c.startswith('ACF_Vul')]

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
        
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats


def collect_plain(pars_post, n_collect=20, coverages=np.linspace(0, 0.8, 33), cxr=True):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()

    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={'PlainACF': {'Coverage': k, 'CXR': cxr}}, t_end=2031)
            
            idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Plain')]

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
    tp = 'PlainACF' if cxr else 'PlainACF_NoCXR'
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [5]:
n_collect = 150
cvs = np.concatenate([np.linspace(0, 0.5, 6), np.linspace(0.5, 1, 11)[1:], np.linspace(1, 2, 11)[1:]])

for folder in folders:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=n_collect, tp='VulACF', coverages=cvs)
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=n_collect, tp='PlainACF', coverages=cvs)
    mss2, mss_stats2 = collect_plain(pars_post, n_collect=n_collect, cxr=False, coverages=cvs)
    mss = pd.concat([mss0, mss1, mss2])
    mss_stats = pd.concat([mss_stats0, mss_stats1, mss_stats2])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_ladd


In [6]:
def collect_fu(pars_post, n_collect=20, coverage=coverage0, tp="VulACF"):
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'VulACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 2}},
        'Vul_2_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 2}},
        'Vul_3_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 3}},
        'Vul_3_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 3}},
    }
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
            
            idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Vulfu') or c.startswith('ACF_Vul')]

            ms = ms[idx].reset_index().assign(Key = i, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [7]:
for folder in folders:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fu(pars_post, n_collect=n_collect, coverage=0.2)
    mss_stats = mss_stats.fillna(0)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_ladd


In [8]:
mss_stats

,Key,Scenario,Pop0,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,ACF_Vul_Sym,ACF_Vul_Vul,...,ACF_VulFu_Sym,ACF_VulFu_Vul,ACF_VulFu_CXR,ACF_VulFu_Xpert,ACF_VulFu_Yield,ACF_VulFu_TP,ACF_VulFu_DS_Fl,ACF_VulFu_DR_Fl,ACF_VulFu_DR_Sl,Coverage
0,15,Baseline,2.383517,0.406623,0.020343,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
1,15,Vul_0_0,2.383517,0.398104,0.019612,4.438170,4.408533,0.324568,4.408533,4.408533,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
2,15,Vul_2_3,2.383517,0.395861,0.019418,4.438325,4.408533,0.324362,4.408533,4.408533,...,2.945977,0.0,0.0,0.109619,0.007921,0.004731,0.004491,0.0,0.000240,0.2
3,15,Vul_2_6,2.383517,0.396830,0.019502,4.438240,4.408533,0.324454,4.408533,4.408533,...,1.475180,0.0,0.0,0.055292,0.004346,0.002749,0.002609,0.0,0.000140,0.2
4,15,Vul_3_3,2.383517,0.395338,0.019370,4.438361,4.408533,0.324311,4.408533,4.408533,...,3.603245,0.0,0.0,0.134165,0.009880,0.005982,0.005679,0.0,0.000303,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,1492,Vul_0_0,2.383080,0.326514,0.016950,4.432014,4.407724,0.330102,4.407724,4.407724,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
896,1492,Vul_2_3,2.383080,0.322629,0.016610,4.432069,4.407724,0.329761,4.407724,4.407724,...,2.971550,0.0,0.0,0.111109,0.008334,0.004944,0.004730,0.0,0.000214,0.2
897,1492,Vul_2_6,2.383080,0.324054,0.016734,4.432016,4.407724,0.329893,4.407724,4.407724,...,1.487791,0.0,0.0,0.056282,0.004780,0.003083,0.002950,0.0,0.000134,0.2
898,1492,Vul_3_3,2.383080,0.321732,0.016527,4.432085,4.407724,0.329678,4.407724,4.407724,...,3.636352,0.0,0.0,0.136062,0.010370,0.006224,0.005955,0.0,0.000269,0.2


In [9]:
def collect_fux(pars_post, n_collect=20, coverage=coverage0, tp="VulACF"):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()

    # Test run
    pars = pars_post[0]
    y1, pars = m.find_baseline(pars, 2022)
    _, ms, _ = m.simulate_onward(y1, pars, intv={'VulACF': {'Coverage': 0.01, 'FollowUp': 1, 'Duration': 1}}, t_end=2025)

    idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Vul')]
    print(idx)
    
    #
    
    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
    
        _, ms, _ = m.simulate_onward(y1, pars, intv={}, t_end=2031)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        ms_stats = {'Key': i, 'Scenario': 'Baseline', 'Pop0': ms.Pop.iloc[0], 
                    'FollowUp': 0, 'Duration': 0}
        for key in idx[1:]:
            try:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            except KeyError:
                ms_stats[key] = 0

        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            if n_fu > 0:
                fu = 1 / n_fu
            else:
                fu = 0
            for dur in np.linspace(0, 4, 9):
                intv = {'VulACF': {'Coverage': coverage, 'FollowUp': fu, 'Duration': dur}}
                
                _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
                try:
                    ms = ms.reset_index().assign(Key = i, FollowUp=fu, Duration=dur)
                    mss.append(ms)

                    ms_stats = {'Key': i, 'Scenario': 'Intervention', 'Pop0': ms.Pop.iloc[0], 
                                'FollowUp': fu, 'Duration': dur}

                    for key in idx[1:]:
                        try:
                            ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
                        except KeyError:
                            ms_stats[key] = 0

                    mss_stats.append(ms_stats)
                except TypeError:
                    pass
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [10]:
for folder in folders:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fux(pars_post, n_collect=n_collect, coverage=0.2)
    mss_stats = mss_stats.fillna(0)
    mss.to_csv(f'{out_path}/Sim_VulACF_fudur.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_fudur_stats.csv')

dy_ladd
['Pop', 'IncR', 'MorR', 'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Sym', 'ACF_Vul_Vul', 'ACF_Vul_CXR', 'ACF_Vul_Xpert', 'ACF_Vul_Yield', 'ACF_Vul_TP', 'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 'ACF_VulFu_Footfall', 'ACF_VulFu_Screened', 'ACF_VulFu_Confirmed', 'ACF_VulFu_Sym', 'ACF_VulFu_Vul', 'ACF_VulFu_CXR', 'ACF_VulFu_Xpert', 'ACF_VulFu_Yield', 'ACF_VulFu_TP', 'ACF_VulFu_DS_Fl', 'ACF_VulFu_DR_Fl', 'ACF_VulFu_DR_Sl']


In [11]:
mss_stats

,Key,Scenario,Pop0,FollowUp,Duration,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,...,ACF_VulFu_Sym,ACF_VulFu_Vul,ACF_VulFu_CXR,ACF_VulFu_Xpert,ACF_VulFu_Yield,ACF_VulFu_TP,ACF_VulFu_DS_Fl,ACF_VulFu_DR_Fl,ACF_VulFu_DR_Sl,Coverage
0,3,Baseline,2.383096,0.000000,0.0,0.261861,0.019515,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
1,3,Intervention,2.383096,0.000000,0.0,0.254347,0.018548,4.426174,4.407755,0.326607,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
2,3,Intervention,2.383096,0.000000,0.5,0.254348,0.018546,4.426177,4.407755,0.326605,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
3,3,Intervention,2.383096,0.000000,1.0,0.254348,0.018546,4.426177,4.407755,0.326605,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
4,3,Intervention,2.383096,0.000000,1.5,0.254348,0.018546,4.426177,4.407755,0.326605,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,1498,Intervention,2.383229,0.166667,2.0,0.161560,0.013067,4.419066,4.408000,0.323924,...,4.520634,0.0,0.0,0.165566,0.006108,0.003612,0.003474,0.0,0.000139,0.2
9596,1498,Intervention,2.383229,0.166667,2.5,0.161251,0.013021,4.419076,4.408000,0.323880,...,5.082247,0.0,0.0,0.186178,0.006949,0.004143,0.003984,0.0,0.000159,0.2
9597,1498,Intervention,2.383229,0.166667,3.0,0.161008,0.012983,4.419085,4.408000,0.323843,...,5.535158,0.0,0.0,0.202781,0.007609,0.004554,0.004380,0.0,0.000174,0.2
9598,1498,Intervention,2.383229,0.166667,3.5,0.160812,0.012953,4.419092,4.408000,0.323814,...,5.907158,0.0,0.0,0.216423,0.008158,0.004898,0.004711,0.0,0.000187,0.2
